Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization, Lambda
from keras.models import model_from_json, Model
import keras.backend as K
from keras import regularizers
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import gen_classifier_dataset

import json
import time

import numpy as np
import sklearn

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 5
BINS_EDGE = np.load("./s1p10_data/s1p10_bins_edge.npy")
NUM_CLASSES = len(BINS_EDGE) - 1  

with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    CLASSES_WEIGHT = json.load(fp)

# 1. Model Acrchitect

1 Image -> ResNet-8 -> Dense regressor -> 5 steering angles

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [3]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
# encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
# for l in encoder.layers:
#     l.trainable = False
# encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## 1.3 Create Decoder
Decoder is a seri of Dense layers with a non-activate Dense layer at the end to perform regression.

## 1.4 Create Model

In [4]:
reshapor = Reshape((1, -1))

def s1p10_model(input_shape):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Dropout(0.5)(X_feature)
    
    # apply classifier body
    X_body = Dense(800, activation='relu')(X_feature)
    
    # apply classifier head
    y = []
    for i in range(NUM_LABELS):
        X_body_1 = Dropout(0.5)(X_body)
        out = Dense(NUM_CLASSES, activation='softmax', name="head_%d" % i)(X_body_1)
        y.append(out)
    
#     # apply classifier head
#     class_neck = []
#     for i in range(NUM_LABELS):
#         X = Dense(300, activation='relu', name="neck_%d" % i)(X_body)
#         X = reshapor(X)
#         class_neck.append(X)
    
#     # apply LSTM
#     X = keras.layers.concatenate(class_neck, axis=1)  # to get shape (None, 5, 300)
#     X = LSTM(128, return_sequences=True, dropout=0.5)(X)
    
#     # apply classifier head
#     y = []
#     for i in range(NUM_LABELS):
#         X_neck = Lambda(lambda X : X[:, i, :])(X)
#         X = Dropout(0.5)(X)
#         out = Dense(NUM_CLASSES, activation="softmax", name="head_%d" % i)(X_neck)
#         y.append(out)
    
    model = Model(inputs=[X_in], outputs=y)
    return model
        

In [5]:
model = s1p10_model(IMAGE_SHAPE)

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 6272)         0           resnet8[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 800)          5018400     dropout_1[0][0]             

In [6]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['acc'])

# Training

In [7]:
gen_param = {'num_classes': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': BINS_EDGE, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': None, 
             'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'flip_prob': 0.5}

# X_train, y_train = gen_classifier_dataset("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
X_train = np.load('./s1p10_data/bottom_half_CH2_training_X.npy')
y_train_arr = np.load('./s1p10_data/bottom_half_CH2_training_y.npy')  # shape (5, len_dataset, num_classes)
y_train = [y_train_arr[i, :, :]for i in range(NUM_LABELS)]

# X_val, y_val = gen_classifier_dataset("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)
X_val = np.load('./s1p10_data/bottom_half_CH2_validation_X.npy')
y_val_arr = np.load('./s1p10_data/bottom_half_CH2_validation_y.npy')
y_val = [y_val_arr[i, :, :]for i in range(NUM_LABELS)]

In [14]:
batch_size = 200

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')
early_stop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              patience=5,
                                              restore_best_weights=True)

# eva_metric = EVAMetrics()

history = model.fit(x=X_train,
                  y=y_train,
                  epochs=50,
                  validation_data=(X_val, y_val),
                  class_weight=CLASSES_WEIGHT,
                  initial_epoch=25,
                  shuffle=True,
                  batch_size=batch_size,
                  callbacks=[tb_callback, early_stop_cb])

# eva_data = eva_metric.get_data()

Train on 18232 samples, validate on 2026 samples
Epoch 26/50
18232/18232 [==============================] - 44s 2ms/step - loss: 2.4116 - head_0_loss: 0.5459 - head_1_loss: 0.4701 - head_2_loss: 0.4522 - head_3_loss: 0.4598 - head_4_loss: 0.4836 - head_0_acc: 0.8096 - head_1_acc: 0.8364 - head_2_acc: 0.8377 - head_3_acc: 0.8330 - head_4_acc: 0.8285 - val_loss: 4.2479 - val_head_0_loss: 0.9375 - val_head_1_loss: 0.8459 - val_head_2_loss: 0.8294 - val_head_3_loss: 0.8204 - val_head_4_loss: 0.8147 - val_head_0_acc: 0.7187 - val_head_1_acc: 0.7404 - val_head_2_acc: 0.7374 - val_head_3_acc: 0.7364 - val_head_4_acc: 0.7483
Epoch 27/50
18232/18232 [==============================] - 40s 2ms/step - loss: 2.3581 - head_0_loss: 0.5242 - head_1_loss: 0.4607 - head_2_loss: 0.4401 - head_3_loss: 0.4547 - head_4_loss: 0.4784 - head_0_acc: 0.8174 - head_1_acc: 0.8378 - head_2_acc: 0.8430 - head_3_acc: 0.8380 - head_4_acc: 0.8296 - val_loss: 4.0768 - val_head_0_loss: 0.9225 - val_head_1_loss: 0.8029 - 

# Save weights

In [15]:
# serialize model to JSON
model_json = model.to_json()
with open(log_dir + "/bottom_half_s1p10_model_%s.json" % time_str, "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(log_dir + "/bottom_half_s1p10_model_%s.h5" % time_str)
print("Saved model to disk")

Saved model to disk


In [10]:
time_str

'2019_06_04_20_13'

In [13]:
# #TODO: save X_train, y_train
# np.save('./s1p10_data/bottom_half_CH2_training_X.npy', X_train)
# np.save('./s1p10_data/bottom_half_CH2_training_y.npy', y_train)

# np.save('./s1p10_data/bottom_half_CH2_validation_X.npy', X_val)
# np.save('./s1p10_data/bottom_half_CH2_validation_y.npy', y_val)